In [15]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='ee-chrismihiar')

In [20]:
county_collection = ee.FeatureCollection("TIGER/2018/Counties") \
    .filter(ee.Filter.eq('STATEFP', '37'))
# filtered_counties = counties.filter(ee.Filter.eq('COUNTYFP', '001'))
    
nasa_collection = ee.ImageCollection('NASA/GDDP-CMIP6') \
    .filter(ee.Filter.date('2014-07-01', '2015-07-02')) \
    .filter(ee.Filter.eq('model', 'ACCESS-CM2')) \
    .filter(ee.Filter.eq('scenario', 'ssp585')) \
    .select('tasmax')

image = nasa_collection.first()

In [34]:
# This function adds a band representing the image timestamp.
def add_time(image):
  return image.addBands(image.getNumber('system:time_start'))

# Map the function over the collection and display the result.
display(nasa_collection.map(add_time))


In [35]:

# Calculate mean values for each county
county_tasmax_means = image.reduceRegions(
        collection=county_collection,
        reducer=ee.Reducer.mean(),
        scale=27830,  # Native resolution for NEX-GDDP-CMIP6
    )

display(county_tasmax_means)


In [ ]:

# Export the FeatureCollection.
task = ee.batch.Export.table.toDrive(
    collection=county_tasmax_means,
    description='exportTableExampleNC',
    fileFormat='CSV',
)
task.start()

In [21]:
# # Load an image and select some bands of interest.
# image = ee.Image('LANDSAT/LC08/C02/T1/LC08_044034_20140318').select(
#     ['B4', 'B3', 'B2']
# )

# Reduce the image to get a one-band maximum value image.
max_value = image.reduce(ee.Reducer.max())
max_value

In [13]:
image = ee.Image(nasa_collection.first())
mean_value = image.reduce(ee.Reducer.mean())
mean_value

In [28]:
image = ee.Image(nasa_collection.first())

# mean_value = image.reduce(ee.Reducer.mean())

m = geemap.Map()
# m.center_object(image, 10)
m.add_layer(image, {'min': 200, 'max': 300}, 'Temperature')
m


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [2]:
# Load an image and select some bands of interest.
image = ee.Image('LANDSAT/LC08/C02/T1/LC08_044034_20140318').select(
    ['B4', 'B3', 'B2']
)

# Reduce the image to get a one-band maximum value image.
max_value = image.reduce(ee.Reducer.max())

# Display the result.
m = geemap.Map()
m.center_object(image, 10)
m.add_layer(max_value, {'max': 13000}, 'Maximum value image')
m

Map(center=[37.47164678275328, -122.14450014746849], controls=(WidgetControl(options=['position', 'transparent…

In [39]:
startDate = ee.Date('2017-01-01')
endDate = ee.Date('2017-01-05')  # Shortened the date range

tmpCollection = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')\
  .select('tmmx', 'tmmn')

nonCONUS = ['02', '15', '60', '66', '69', '72', '78']
counties = ee.FeatureCollection('TIGER/2018/Counties').filter(
    ee.Filter.inList('STATEFP', nonCONUS)).limit(2)  # Just picked two counties

#print(counties.size(), 'counties')
# visualize        
m.addLayer(counties, {}, 'counties')
m


Map(bottom=3376.0, center=[40.84706035607122, -91.64794921875001], controls=(WidgetControl(options=['position'…

In [45]:
days = endDate.difference(startDate, "days")
dailyTempByCounty = ee.FeatureCollection(ee.List.sequence(0, days.subtract(1))
  .map(lambda dayOffset: 
    # This function needs to return a FeatureCollection for each day
    ee.FeatureCollection(counties.map(lambda county: 
      # For each day and county, create a feature with temperature data
      ee.Feature(county.geometry())
        .set(tmpCollection
          .filterDate(
            startDate.advance(ee.Number(dayOffset), 'days'), 
            startDate.advance(ee.Number(dayOffset).add(1), 'days')
          )
          .mean()
          .reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=county.geometry(),
            scale=4000
          ))
        .set('date', startDate.advance(ee.Number(dayOffset), 'days').format('yyyy-MM-dd'))
    ))
  )).flatten()

print(dailyTempByCounty)

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.flatten",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection",
          "arguments": {
            "features": {
              "functionInvocationValue": {
                "functionName": "List.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_2_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Collection.map",
                          "arguments": {
                            "baseAlgorithm": {
                              "functionDefinitionValue": {
                                "argumentNames": [
                                  "_MAPPING_VAR_1_0"
                          